In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms
from PIL import Image

def load_pretrained_model():
    """Load a pretrained ResNet-50 model in evaluation mode."""
    model = models.resnet50(pretrained=True)
    model.eval()
    return model

def preprocess_image(image_path):
    """Preprocess the image: resize, crop, normalize, and convert to tensor."""
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image).unsqueeze(0).requires_grad_(True)
    return image, input_tensor

def get_prediction(model, input_tensor):
    """Perform forward pass and return the predicted class index."""
    output = model(input_tensor)
    predicted_class = output.argmax().item()
    return output, predicted_class

In [ ]:
if __name__ == "__main__":
    model = load_pretrained_model()
    image_path = "Ressources/panda.jpg"
    image, input_tensor = preprocess_image(image_path)
    output, predicted_class = get_prediction(model, input_tensor)

In [9]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms
from PIL import Image

In [2]:
model = models.resnet50(pretrained=True)
model.eval()

c:\Users\nasta\miniconda3\envs\adaexam\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nasta\miniconda3\envs\adaexam\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
no_of_layers = 0
conv_layers = []

model_children = list(model.children())

def extract_conv_layers(model):
    global no_of_layers
    for layer in model.children():
        if isinstance(layer, nn.Conv2d):
            no_of_layers += 1
            conv_layers.append(layer)
        elif isinstance(layer, nn.Sequential) or isinstance(layer, nn.Module):
            extract_conv_layers(layer)

extract_conv_layers(model)

In [ ]:
print("nomber of layers: ", no_of_layers)
conv_layers

nomber of layers:  53


[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
 Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
 Conv2d(256, 

In [21]:
def preprocess_image(image_path):
    """Preprocess the image: resize, crop, normalize, and convert to tensor."""
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    ])
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0)
    return image

image_path = "Ressources/panda.jpg"
image = preprocess_image(image_path)

image.shape

torch.Size([1, 3, 224, 224])

In [ ]:
import torch.nn.functional as F

results = [image]
current_input = image  

for i in range(no_of_layers):
    try:
        new_input = conv_layers[i](current_input)
        current_input = new_input
        results.append(current_input)
    except RuntimeError as e:
        print(f"Layer {i} skipped due to shape mismatch: {e}")
        new_input = new_input + current_input
        #continue # skips when there is skip connection
        current_input = new_input
        results.append(current_input)


for i, res in enumerate(results):
    print(f"Layer {i}: {res.shape}")

Layer 4 skipped due to shape mismatch: Given groups=1, weight of size [256, 64, 1, 1], expected input[1, 256, 112, 112] to have 64 channels, but got 256 channels instead
Layer 14 skipped due to shape mismatch: Given groups=1, weight of size [512, 256, 1, 1], expected input[1, 512, 56, 56] to have 256 channels, but got 512 channels instead
Layer 27 skipped due to shape mismatch: Given groups=1, weight of size [1024, 512, 1, 1], expected input[1, 1024, 28, 28] to have 512 channels, but got 1024 channels instead
Layer 46 skipped due to shape mismatch: Given groups=1, weight of size [2048, 1024, 1, 1], expected input[1, 2048, 14, 14] to have 1024 channels, but got 2048 channels instead
Layer 0: torch.Size([1, 3, 224, 224])
Layer 1: torch.Size([1, 64, 112, 112])
Layer 2: torch.Size([1, 64, 112, 112])
Layer 3: torch.Size([1, 64, 112, 112])
Layer 4: torch.Size([1, 256, 112, 112])
Layer 5: torch.Size([1, 256, 112, 112])
Layer 6: torch.Size([1, 64, 112, 112])
Layer 7: torch.Size([1, 64, 112, 11

In [ ]:
import math
import torch
import matplotlib.pyplot as plt

for num_layer in range(len(results)):
    plt.figure(figsize=(16, 16))

    layer_viz = results[num_layer].squeeze()
    print(f"Layer {num_layer + 1}: {layer_viz.shape}")

    num_filters = layer_viz.shape[0]
    grid_size = math.ceil(math.sqrt(num_filters))

    for i, feature_map in enumerate(layer_viz):
        plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(feature_map.detach().cpu().numpy())
        plt.axis("off")

    plt.tight_layout()
    plt.show()
    plt.close()
